In [ ]:
%%sh
pip install -U --no-build-isolation --no-deps ../input/pytorch-timm/pytorch-image-models-master/ -qq

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

import os
import cv2
import timm
from timm.models import load_checkpoint

from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader

import gc
import wandb
import warnings
from tqdm.notebook import tqdm

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

warnings.simplefilter('ignore')

In [ ]:
Config = {
    'SVR_TRAIN': True,
    'CSV_PATH': "../input/petfinder-pawpularity-score/test.csv",
    'IMG_PATH': "../input/petfinder-pawpularity-score/test",
    'N_ACCUM': 2,
    'N_SPLITS': 5,
    'TRAIN_BS': 64,
    'VALID_BS': 64,
    'N_EPOCHS': 5,
    'NUM_WORKERS': 4,
    'LR': 1e-5,
    'OPTIM': "AdamW",
    'LOSS': "BCELogits",
    'ARCH': "swin_large_patch4_window12_384_in22k",
    'IMG_SIZE': 384,
    'DEVICE': "cuda",
    "T_0": 20,
    "η_min": 1e-4,
    'infra': "Kaggle",
    'competition': 'petfinder',
    '_wandb_kernel': 'tanaym',
    "wandb": False,
}

In [ ]:
def rmse(output, target):
    """
    Returns root mean squared error loss
    """
    return mean_squared_error(output, target, squared=False)

In [ ]:
class PetfinderData(Dataset):
    def __init__(self, df, config=Config, augments=None, is_test=False):
        self.df = df
        self.augments = augments
        self.is_test = is_test
        self.config = config
        
        self.img_paths = self._get_img_paths(self.df, self.config)
        self.meta_feats = self._get_meta_feats(self.df, self.is_test)

    def __getitem__(self, idx):
        img = cv2.imread(self.img_paths[idx])
        img = cv2.resize(img, (Config['IMG_SIZE'], Config['IMG_SIZE']))
        meta_feats = torch.tensor(self.meta_feats.iloc[idx].values).float()

        if self.augments:
            img = Image.fromarray(img)
            img = self.augments(img)
        
        if self.is_test:
            return (img, meta_feats)
        else:
            target = torch.tensor(self.df['Pawpularity'].iloc[idx]).float()
            return (img, meta_feats, target)
    
    def __len__(self):
        return len(self.df)

    def _get_img_paths(self, df, config):
        """
        Returns the image paths in a list
        """
        imgs = df['Id'].apply(lambda x: os.path.join(config['IMG_PATH'], x + ".jpg")).tolist()
        return imgs
    
    def _get_meta_feats(self, df, is_test):
        """
        Returns the meta features in a df
        """
        if self.is_test:
            meta = self.df.drop(['Id'], axis=1)
            return meta
        else:
            meta = self.df.drop(['Id', 'Pawpularity'], axis=1)
            return meta

In [ ]:
class RegressionHeadModel(nn.Module):
    def __init__(self, backbone_arch, pretrained= False, in_chans=3):
        super(RegressionHeadModel, self).__init__()
        self.backbone = timm.create_model(backbone_arch, pretrained=pretrained)
        self.backbone.head = nn.Linear(self.backbone.head.in_features, 128)
        self.drop = nn.Dropout(0.3)
        self.fc1 = nn.Linear(140, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, img, meta):
        emb = self.backbone(img)
        x = self.drop(emb)
        x = torch.cat([x, meta], dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        concatenated = torch.cat([x, emb, meta], dim=1)
        return concatenated

In [ ]:
def mixup_augmentation(x:torch.Tensor, y:torch.Tensor, alpha:float = 1.0):
    """
    Function which performs Mixup augmentation
    """
    assert alpha > 0, "Alpha must be greater than 0"
    assert x.shape[0] > 1, "Need more than 1 sample to apply mixup"

    lam = np.random.beta(alpha, alpha)
    rand_idx = torch.randperm(x.shape[0])
    mixed_x = lam * x + (1 - lam) * x[rand_idx, :]

    target_a, target_b = y, y[rand_idx]

    return mixed_x, target_a, target_b, lam

In [ ]:
class Augments:
    IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
    IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB
    train_augments = T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ToTensor(),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        )
    valid_augments = T.Compose(
            [
                T.ToTensor(),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        )

In [ ]:
class SVRModule:
    def __init__(self, config, dataloader, model, device="cuda:0", apex=False):
        self.test_loader= dataloader
        self.model = model
        self.device = torch.device(device)
        self.config = config


    @torch.no_grad()
    def predict(self, model_name):
        """ 
        SVR Validation
        """
        
        clf = pickle.load(open(model_name, "rb"))
        self.model.eval()
        test_pbar = tqdm(enumerate(self.test_loader), total=len(self.test_loader))
        test_preds = []
        test_embeddings = []

        for idx, cache in test_pbar:
            img = self._convert_if_not_tensor(cache[0], dtype=torch.float32)
            meta = self._convert_if_not_tensor(cache[1], dtype=torch.float32)
            output = self.model(img, meta).squeeze()

            embedding = output[:, 1:]
            output = output[:, :1]
            
            output=output.sigmoid() * 100
            
            test_preds += [output.cpu().numpy()]
            test_embeddings += [embedding.cpu().numpy()]
            
        all_test_preds = np.concatenate(test_preds)
        all_test_embeddings = np.concatenate(test_embeddings)
        
        all_test_SVR_prediction = clf.predict(all_test_embeddings)
        
        ensemble_prediction = (all_test_SVR_prediction + all_test_preds.reshape(-1))/2.0
        # Tidy
        del img, meta, test_preds, test_embeddings, all_test_preds, all_test_embeddings, output, embedding, all_test_SVR_prediction
        gc.collect()
        torch.cuda.empty_cache()
        
        return ensemble_prediction

    def _convert_if_not_tensor(self, x, dtype):
        if self._tensor_check(x):
            return x.to(self.device, dtype=dtype)
        else:
            return torch.tensor(x, dtype=dtype, device=self.device)

    def _tensor_check(self, x):
        return isinstance(x, torch.Tensor)

In [ ]:
if __name__ == '__main__':
    test_file = pd.read_csv(Config['CSV_PATH'])    
    submission_df=pd.DataFrame(columns=["Id","Pawpularity"])
    submission_df["Id"]=test_file["Id"]
    for fold in range(Config["N_SPLITS"]): 
        test_set = PetfinderData(
            df = test_file,
            config = Config,
            is_test=True,
            augments=Augments.valid_augments)


        test_loader = DataLoader(
            test_set,
            batch_size = Config['TRAIN_BS'],
            shuffle = False,
            num_workers = Config['NUM_WORKERS'],
            pin_memory = True
        )

        model = RegressionHeadModel(backbone_arch=Config['ARCH'])
        model = model.to(torch.device(Config['DEVICE']))

        model.load_state_dict(torch.load(f"../input/pwp-swintransformer/models/swin_large_patch4_window12_384_in22k_{fold}_model.bin"))

        svrmodule = SVRModule(
            config = Config,
            dataloader=test_loader,
            model = model,
            apex=True
        )

        fold_pred = svrmodule.predict(f'../input/pwp-svr/swin_large_patch4_window12_384_in22k_SVR_{fold}.pkl')

        submission_df[fold] = fold_pred
    submission_df["Pawpularity"]=(submission_df[0] +submission_df[1] +submission_df[2] +submission_df[3] +submission_df[4])/5.0
    submission_df[["Id","Pawpularity"]].to_csv("submission.csv", index=None)
    submission_df.head()

In [ ]:
submission_df.head()